## <center>Разметки новостной ленты </center>

Для улучшения предсказания временных рядов, курса валют или акций, полезно прибегнуть к ансамблю моделей, одна из которцх построена на LSTM (пример https://github.com/BlackStoneShadow/Python/blob/main/MLearning/ANN/lesson8/Task1.ipynb), другая учитывает новостной фон (https://github.com/BlackStoneShadow/Python/tree/main/MLearning/Markup/Lesson5/Task1 - реализация получение нововстной ленты), такая совместная работа моделей позволит выполнять более взвешенные предсказания значений.

In [4]:
pip install pandas

In [5]:
pip install scikit-learn

In [6]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: regex>=2021.8.3 in w:\python\mlearning\markup\lesson9\task1\env32\lib\site-packages (from nltk) (2023.8.8)



In [7]:
pip install TextBlob

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from textblob import TextBlob

import pandas as pd
import numpy as np
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Protorsky\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [78]:
positive = ['рост', 'стабильность', 'здорово', 'надежда', 'победа', 'подорожали', 'высокая доходность']
negative = ['падение', 'кризис', 'рецессия', 'давление', 'беспокойство', 'приостановлены', 'арест', 'инфляционные ожидания']

def get_class(text):
    words = nltk.word_tokenize(text.lower())
    
    count_pos = sum([1 for word in words if word in positive])
    count_neg = sum([1 for word in words if word in negative])
    
    if count_pos > count_neg:
        return 'Positive'
    elif count_pos < count_neg:
        return 'Negative'
    else:
        return 'Neutral'

In [79]:
print(get_class('рост рынка'))
print(get_class('падение рынка'))
print(get_class('прогноз рынка'))

Positive
Negative
Neutral


In [80]:
def get_polarity(text): 
    blob = TextBlob(text) 
    return blob.sentiment.polarity

In [81]:
# механизм полярности не работает
print(get_polarity('рост рынка'))
print(get_polarity('падение рынка'))
print(get_polarity('прогноз рынка'))

0.0
0.0
0.0


In [82]:
news = pd.read_csv('news.csv', engine='python', encoding='utf-8', error_bad_lines=False)

news.head(10)

,date,article,text
0,2024-04-08 00:00:00,Борьба Мосбиржи с «несправедливыми IPO»: новос...,Investing.com — Мосбиржа внедрит сервис для бо...
1,2024-04-08 00:00:00,Главные новости: самолеты Boeing продолжают ра...,Investing.com — На этой неделе начнется новый ...
2,2024-04-08 00:00:00,«Конгрессу надо действовать быстро»: эксперт п...,Investing.com — Из-за высокого давления госдол...
3,2024-04-08 00:00:00,"США пытаются наладить отношения с Китам, но ес...",Investing.com — Министр финансов США Джанет Йе...
4,2024-04-09 00:00:00,Взрывной рост Toncoin: новости к утру 9 апреля,Investing.com — Toncoin обновил исторический м...
5,2024-04-09 00:00:00,Инфляционные ожидания в США стабилизируются,Investing.com — Ожидания американских потребит...
6,2024-04-09 00:00:00,Главные новости: выступления спикеров ФРС и но...,Investing.com — В преддверии выхода ключевых д...
7,2024-03-31 00:00:00,Топ-5 событий на этой неделе: экономданные из ...,Investing.com — Данные по занятости в США в пя...
8,2024-03-17 00:00:00,Топ-5 недели: в центре внимания - решения цент...,Investing.com — Решения центральных банков буд...
9,2024-03-18 00:00:00,Явная победа Путина на выборах: новости к утру...,"Investing.com — Путин набирает 87,33% на выбор..."


In [83]:
# выборка для выполнения разметки
train_news = news.sample(frac = 0.1)
print(f'length train={len(train_news)}')
# перемешиваем для сохранения распределения
train_news = shuffle(train_news)
# разметка на основе своих правил
train_news['sentiment_label'] = train_news['text'].apply(get_class)

length train=100


In [84]:
def train_model(labeled_news):     
    vectorizer = TfidfVectorizer()
    # векторизация текстовых данных с помощью TF-IDF 
    X = vectorizer.fit_transform(labeled_news['text']) 
    y = labeled_news['sentiment']
    # Обучение модели логистической регрессии на размеченных данных 
    model = LogisticRegression() 
    model.fit(X, y)
    
    return model, vectorizer

In [85]:
# выполним label-encoding
encoder = LabelEncoder().fit(train_news['sentiment_label'])
train_news['sentiment'] = encoder.transform(train_news['sentiment_label'])
# делим набор для начала активного обучения
labeled_news, unlabeled_news = train_test_split(train_news, train_size=0.2, random_state=42)
# обучение модели
model, vectorizer = train_model(labeled_news)
# предсказание
X_unlabeled = vectorizer.transform(unlabeled_news['text']) 
y_unlabeled_predicted = model.predict(X_unlabeled)
# определение энтропии данных
y_unlabeled_proba = model.predict_proba(X_unlabeled) 
uncertainty = -(y_unlabeled_proba * np.log2(y_unlabeled_proba)).sum(axis=1)
# поиск записей требующих разметки человеком
labeled_news_new = unlabeled_news.iloc[uncertainty.argsort()[:20]][['date','article','text','sentiment_label']]
# требуется ручная разметка
labeled_news_new.to_csv('uncertainty.csv')
labeled_news_new.head(10)

,date,article,text,sentiment_label
849,2023-07-29 00:00:00,Совфед одобрил профилактические проверки бизне...,"МОСКВА, 29 июл -- ПРАЙМ. Сенаторы одобрили зак...",Neutral
950,2023-07-29 00:00:00,"Совфед разрешил ""Ростеху"" напрямую выпускать о...","МОСКВА, 28 июл -- ПРАЙМ. Сенаторы одобрили зак...",Neutral
972,2023-07-30 00:00:00,Мишустин упростил онлайн-торговлю Калининграда...,"МОСКВА, 30 июл -- ПРАЙМ. Премьер-министр РФ Ми...",Neutral
940,2023-07-29 00:00:00,Кабмин утвердил новый порядок формирования и н...,"МОСКВА, 28 июл -- ПРАЙМ. Правительство утверди...",Neutral
902,2023-07-29 00:00:00,В OneTwoTrip рассказали о самых дорогих зарубе...,"МОСКВА, 29 июл -- ПРАЙМ. Самое дорогое зарубеж...",Neutral
772,2023-08-08 00:00:00,Генслер считает ИИ эпицентром грядущего кризиса,Investing.com — Глава Комиссии по ценным бумаг...,Neutral
977,2023-07-30 00:00:00,Медведев посоветовал врагам молиться на россий...,"МОСКВА, 30 июл -- ПРАЙМ. Врагам России следует...",Neutral
836,2023-08-03 00:00:00,Минфин объявил о переходе к покупкам валюты вп...,Investing.com — Минфин представил ежемесячный ...,Neutral
291,2023-12-07 00:00:00,Страны G7 договорились о запрете на импорт алм...,Investing.com — Страны «Большой семерки» (Герм...,Neutral
818,2023-07-31 00:00:00,Власти Белоруссии призвали заводы осваивать ро...,"МИНСК, 30 июл -- ПРАЙМ. Вице-премьер Белорусси...",Neutral


![Markup LabelStudio](./label.jpg "LabelStudio")

In [103]:
# загрузка после ручной разметки
labeled_news_new = pd.read_csv('labeled.csv', engine='python', encoding='utf-8', error_bad_lines=False)
labeled_news_new = labeled_news_new[['date','article','text','sentiment']]
# выполним label-encoding
labeled_news_new['sentiment_label'] = labeled_news_new['sentiment']
labeled_news_new['sentiment'] = encoder.transform(labeled_news_new['sentiment_label'])
# объединение наборов
labeled_news = pd.concat([labeled_news, labeled_news_new])
# не забываем перемешать
labeled_news = shuffle(labeled_news)
# делим набор на обучающий и тестовый
train_news, test_news = train_test_split(labeled_news, train_size=0.2, random_state=80)
# переобучение модели на полных данных
model, vectorizer = train_model(train_news)

In [104]:
# расчитываем метрику
X_test = vectorizer.transform(test_news['text']) 
y_test_predicted = model.predict(X_test) 

f1 = f1_score(np.array(test_news['sentiment']), y_test_predicted, average='macro')

print(f'f1={f1}')

f1=0.614646744143147


<center> поскольку у нас три класса используем averago='micro' </center>

![averago='micro'](./micro.png "averago=micro")

расчитанная метрика слабая - необходимо улучшение правил разметки и повышение количества строк обработанных человеком, требуется уеличение обучающего набора.

In [105]:
# формируем итоговый набор с предсказанными классами
X = vectorizer.transform(news['text']) 
y_predicted = model.predict(X) 

news = pd.DataFrame({'date': news['date'], 'article':news['article'], 'text':news['text'], 'sentiment_label': encoder.inverse_transform(y_predicted)})

news.head(10)

,date,article,text,sentiment_label
0,2024-04-08 00:00:00,Борьба Мосбиржи с «несправедливыми IPO»: новос...,Investing.com — Мосбиржа внедрит сервис для бо...,Positive
1,2024-04-08 00:00:00,Главные новости: самолеты Boeing продолжают ра...,Investing.com — На этой неделе начнется новый ...,Positive
2,2024-04-08 00:00:00,«Конгрессу надо действовать быстро»: эксперт п...,Investing.com — Из-за высокого давления госдол...,Positive
3,2024-04-08 00:00:00,"США пытаются наладить отношения с Китам, но ес...",Investing.com — Министр финансов США Джанет Йе...,Negative
4,2024-04-09 00:00:00,Взрывной рост Toncoin: новости к утру 9 апреля,Investing.com — Toncoin обновил исторический м...,Positive
5,2024-04-09 00:00:00,Инфляционные ожидания в США стабилизируются,Investing.com — Ожидания американских потребит...,Positive
6,2024-04-09 00:00:00,Главные новости: выступления спикеров ФРС и но...,Investing.com — В преддверии выхода ключевых д...,Positive
7,2024-03-31 00:00:00,Топ-5 событий на этой неделе: экономданные из ...,Investing.com — Данные по занятости в США в пя...,Negative
8,2024-03-17 00:00:00,Топ-5 недели: в центре внимания - решения цент...,Investing.com — Решения центральных банков буд...,Positive
9,2024-03-18 00:00:00,Явная победа Путина на выборах: новости к утру...,"Investing.com — Путин набирает 87,33% на выбор...",Positive


In [89]:
news.to_csv('result.csv')